In [9]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline
import numpy as np
import sympy
import scipy.sparse.linalg
import scipy.sparse as sps
from math import pi

import setup_grids
from core.constit import fourth_order_tensor, second_order_tensor
from core.bc import bc
from fvdiscr import mpsa, fvutils, mpfa
from viz import cell_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
# Homogeneous media

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 5
pert = 0.
### End of parameter definitions

# Permeability tensor, scalar for simplicity
mu = 1
lmbda = 1
alpha = 1

if True:
    rho_list = np.array([1, 1e-2, 1e-4, 1e-6])
    tau_list = np.array([1, 1e-2, 1e-4, 1e-6])
else:
    rho_list = np.array([1])
    tau_list = np.array([1])

# Analytical solution
x, y = sympy.symbols('x y')
ux = sympy.sin(2 * pi * x) * y * (1 - y)
uy =  sympy.sin(2 * pi * x) * sympy.sin(2 * pi * y)
ux_f = sympy.lambdify((x, y), ux, 'numpy')
uy_f = sympy.lambdify((x, y), uy, 'numpy')
dux_x = sympy.diff(ux, x)
dux_y = sympy.diff(ux, y)
duy_x = sympy.diff(uy, x)
duy_y = sympy.diff(uy, y)
divu = dux_x + duy_y
divu_f = sympy.lambdify((x, y), divu, 'numpy')

sxx = 2 * mu * dux_x + lmbda * divu
sxy = mu * (dux_y + duy_x)
syx = mu * (duy_x + dux_y)
syy = 2 * mu * duy_y + lmbda * divu

sxx_f = sympy.lambdify((x, y), sxx, 'numpy')
sxy_f = sympy.lambdify((x, y), sxy, 'numpy')
syx_f = sympy.lambdify((x, y), syx, 'numpy')
syy_f = sympy.lambdify((x, y), syy, 'numpy')

rhs_x = sympy.diff(sxx, x) + sympy.diff(syx, y)
rhs_y = sympy.diff(sxy, x) + sympy.diff(syy, y)
rhs_x_f = sympy.lambdify((x, y), rhs_x, 'numpy')
rhs_y_f = sympy.lambdify((x, y), rhs_y, 'numpy')

u = sympy.sin(2 * pi * x) * y * (1 - y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')


def run_convergence(grid_type):
    num_tau = tau_list.size
    num_rho = rho_list.size
    d_err = np.zeros(num_refs)
    stress_err = np.zeros(num_refs)
    flux_err = np.zeros(num_refs)
    p_err = np.zeros(num_refs)
    eup = np.zeros((num_rho, num_tau))
    esigma = np.zeros((num_rho, num_tau))

    for itergrid, g in enumerate(setup_grids.grid_sequence(basedim, num_refs, grid_type, pert)):
        print(g.num_cells)
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        perm = second_order_tensor.SecondOrderTensor(2, np.ones(g.num_cells))
        
        
        # Permeability tensor
        mu_c = mu * np.ones(g.num_cells)
        lmbda_c = lmbda * np.ones(g.num_cells)
        c_el = fourth_order_tensor.FourthOrderTensor(2, mu_c, lmbda_c)

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.get_boundary_faces()
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        flux, bound_flux = mpfa.mpfa(g, perm, bound_cond)
        div_flow = fvutils.scalar_divergence(g)
        
        a_flow = div_flow * flux

        stress, bound_stress, grad_p, div_d, stabilization = mpsa.biot(g, c_el, bound_cond)
        div_mech = fvutils.vector_divergence(g)
        a_mech = div_mech * stress

        
        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros((g.dim, g.num_faces))
        u_bound_flow = np.zeros(g.num_faces)
        u_bound[0, bound_faces] = ux_f(xf[0, bound_faces], xf[1, bound_faces])
        u_bound[1, bound_faces] = uy_f(xf[0, bound_faces], xf[1, bound_faces])
        u_bound_flow[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces])
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        rhs_mech = np.vstack((rhs_x_f(xc[0], xc[1]), rhs_y_f(xc[0], xc[1]))) * g.cell_volumes
        rhs_flow = rhs_f(xc[0], xc[1]) * g.cell_volumes
        
        rhs_gradp = -np.vstack((dux_f(xc[0], xc[1]), duy_f(xc[0], xc[1]))) * g.cell_volumes
        
        rhs_divd = divu_f(xc[0], xc[1]) * g.cell_volumes
        
        for iter1, rho in enumerate(rho_list):
            for iter2, tau in enumerate(tau_list):
                compr = rho * sps.diags(g.cell_volumes, 0)

                a_biot = sps.bmat([[a_mech, alpha * grad_p],
                           [alpha * div_d, tau * a_flow + compr + stabilization]]).tocsr()

                b = np.hstack((rhs_mech.ravel('F') - alpha * rhs_gradp.ravel('F'),
                               alpha * rhs_divd + tau * rhs_flow + compr * u_f(xc[0], xc[1])))

                # Solve system, derive fluxes
                u_num = scipy.sparse.linalg.spsolve(a_biot, b)

                num_mech = g.dim * g.num_cells

                # Derive stresses and fluxes
                stress_num = stress * u_num[:num_mech] - bound_stress * u_bound.ravel('F')
                flux_num = flux * u_num[num_mech:]

                # Split solution into components
                ux_num = u_num[:num_mech:2]
                uy_num = u_num[1:num_mech:2]
                p_num = u_num[num_mech:]
                stress_x_num = stress_num[::2]
                stress_y_num = stress_num[1::2]

                # Exact solution
                ux_ex = ux_f(xc[0], xc[1])
                uy_ex = uy_f(xc[0], xc[1])
                u_ex = np.vstack((ux_ex, uy_ex))
                u_diff = np.vstack((ux_num - ux_ex, uy_num - uy_ex))

                sx_ex_faces = np.vstack((sxx_f(xf[0], xf[1]), sxy_f(xf[0], xf[1])))
                sy_ex_faces = np.vstack((syx_f(xf[0], xf[1]), syy_f(xf[0], xf[1])))

                stress_x_ex = np.sum(g.face_normals * sx_ex_faces, axis=0) 
                stress_y_ex = np.sum(g.face_normals * sy_ex_faces, axis=0)
                stress_diff = np.vstack((stress_x_num - stress_x_ex, 
                                         stress_y_num - stress_y_ex))
                stress_ex = np.vstack((stress_x_ex, stress_y_ex))

                d_err_this = (np.sqrt(np.sum(g.cell_volumes * u_diff**2)) /
                             np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
                stress_err_this= (np.sqrt(np.sum((g.face_areas ** g.dim) * stress_diff**2))/
                                np.sqrt(np.sum((g.face_areas ** g.dim) * stress_ex**2)))

                p_ex = u_f(xc[0], xc[1])
                du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
                flux_ex = -np.sum(g.face_normals * du_ex_faces, axis=0)
                flux_diff = flux_num - flux_ex

                p_err_this = (np.sqrt(np.sum(g.cell_volumes * (p_num - p_ex)**2)) /
                             np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
                flux_err_this = (np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                                np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
                
                eup[iter1, iter2] = d_err_this + p_err_this * tau
                
                if iter1 == 0 and iter2 == 0:
                    d_err[itergrid] = d_err_this
                    stress_err[itergrid] = stress_err_this
                    flux_err[itergrid] = flux_err_this
                    p_err[itergrid] = p_err_this

    return p_err, flux_err, d_err, stress_err, eup

grids = ['cart', 'triangular']
# grids = ['cart']

for gr in grids:
    u, f, d, s, eup = run_convergence(gr)
    print(u)
    print(f)
    print(d)
    print(s)
    print(eup)


[autoreload of fvdiscr.mpfa failed: Traceback (most recent call last):
  File "C:\Users\keile\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'profile' is not defined
]
[autoreload of fvdiscr.mpsa failed: Traceback (most recent call last):
  File "C:\Users\keile\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'profile' is not defined
]


16
64
256
1024
4096
[ 0.04219824  0.00940904  0.002283    0.00056426  0.00014032]
[ 0.08641344  0.01854657  0.00448522  0.00109056  0.00026678]
[ 0.39475088  0.09583297  0.02453352  0.00622648  0.00156474]
[ 0.15990682  0.05770037  0.01899805  0.00601498  0.00194903]
[[ 0.00170506  0.00153389  0.0014624   0.00146558]
 [ 0.00170787  0.00161675  0.00363846  0.00194904]
 [ 0.0017079   0.00163945  0.00245814  0.0019296 ]
 [ 0.0017079   0.00163971  0.00241167  0.0019294 ]]
32
128
512
2048
8192
[  1.75755710e-02   3.85588207e-03   9.54155997e-04   2.40259723e-04
   6.04108433e-05]
[ 0.12253434  0.03467007  0.00925557  0.00237698  0.00060042]
[ 0.23016805  0.07513434  0.02207607  0.00589821  0.00151175]
[ 0.14445322  0.06201035  0.02203943  0.0069544   0.0021842 ]
[[ 0.00157216  0.00152964  0.00152017  0.00143801]
 [ 0.00157317  0.0017136   0.00222728  0.00159417]
 [ 0.00157318  0.00173464  0.00265976  0.00159277]
 [ 0.00157318  0.00173487  0.00249245  0.00159276]]


In [14]:
import pdb
pdb.pm()

> c:\users\keile\anaconda3\lib\site-packages\numpy\core\shape_base.py(230)vstack()
-> return _nx.concatenate([atleast_2d(_m) for _m in tup], 0)
(Pdb) rhs_mech.ravel('F') - rhs_gradp.ravel('F')
*** NameError: name 'rhs_mech' is not defined
(Pdb) u
> <ipython-input-13-e6199a37566b>(117)run_convergence()
-> rhs_divd + tau * rhs_flow + rho * compr * u_f(xc[0], xc[1])))
(Pdb) rhs_mech.ravel('F') - rhs_gradp.ravel('F')
array([ 0.03356742,  0.02693757, -0.03678921,  0.12155864, -0.08937686,
        0.15021072, -0.18600957,  0.22098221,  0.04591672, -0.01369693,
       -0.04474311,  0.11202919, -0.07166937,  0.10724219, -0.11034574,
        0.11995111,  0.02855738, -0.03325952, -0.0316239 ,  0.05716888,
       -0.06702836,  0.09068139, -0.11526219,  0.12171393,  0.02516367,
       -0.06268138, -0.02251078,  0.02371063, -0.06057947,  0.07944235,
       -0.08520991,  0.0896777 ])
(Pdb) rhs_divd + tau * rhs_flow + rho * compr * u_f(xc[0], xc[1])
array([ 0.08182709,  0.13043798,  0.1418068 ,  0.22